# Session 1: AI Concepts Visualized
## 3. Text Generation with Markov Chains

### Exercise 1: The Curse of Dimensionality
This exercise demonstrates why "distance" becomes meaningless in high-dimensional spaces (like the embedding space of an LLM). As we add dimensions, all points become roughly equidistant, making clustering difficult.

In [1]:
%matplotlib widget
import string
import random
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import Counter

## Exercise 3: Text Generation with Markov Chains
## "Thinking" vs. "Predicting Next Token"

A **Markov Chain** is a "Small Language Model" that predicts the next word based purely on the frequency of previous word sequences.

Before the era of Transformers and ChatGPT, these probabilistic models were the standard for text generation. While modern Large Language Models (LLMs) are infinitely more complex, they share a fundamental root with Markov Chains: they are **probabilistic predictors**, not "thinking" machines.

### 1. How It Works (The Math)

Fundamentally, language modeling is about calculating the probability of the next word $w_t$ given a history of previous words.

#### The Chain Rule
In a perfect world, to predict the next word, we would look at **every** word that came before it:
$$P(w_t | w_{t-1}, w_{t-2}, \dots, w_1)$$

#### The Markov Assumption
However, tracking the *entire* history is computationally impossible for simple models. A Markov Chain simplifies this by assuming that the next word depends **only** on the last $N$ words (the current state).

Formally, an $N$-th order Markov model approximates the probability as:
$$P(w_t | \text{History}) \approx P(w_t | w_{t-1}, w_{t-2}, \dots, w_{t-n+1})$$



### 2. The Process

1.  **The Corpus:** We feed the model a text (e.g., *Sherlock Holmes*).
2.  **Training (Building the Matrix):** The model counts how often word $B$ follows word $A$.
    * *Context:* "The quick brown..."
    * *Probabilities:* $P(\text{fox} | \text{quick brown}) = 0.9$, $P(\text{dog} | \text{quick brown}) = 0.1$.
3.  **Generation:** When asked to write, the model essentially rolls a weighted die based on these probabilities to select the next token.

### 3. The "Memory" Parameter ($N$-Grams)

The variable $N$ defines how much context the model can "see" when calculating probabilities.

* **$N=1$ (Unigram / 0-order):** The model has no memory. It picks words based on global frequency alone.
    * *Result:* "The of and to a..." (Word salad).
* **$N=2$ (Bigram / 1st-order):** The model looks only at the **current** word to guess the next.
    * *Math:* $P(w_t | w_{t-1})$
    * *Result:* "The cat sat on the moon is blue." (Grammatically okay locally, but nonsensical globally).
* **$N=3$ (Trigram / 2nd-order):** The model looks at the last **2** words.
    * *Math:* $P(w_t | w_{t-1}, w_{t-2})$
    * *Result:* "The cat sat on the mat." (Coherent sentences appear).

### 4. Why This Matters: The "Hallucination" Trap

This simple demo exposes the root cause of AI "Hallucinations."

The model does not know facts. It does not know what a "cat" is. It simply knows that statistically, maximizing $P(\text{sat} | \text{cat})$ yields a high score.

If the training data contains misconceptions, or if the most *statistically probable* sequence of words happens to be factually incorrect, the model will generate a lie with 100% confidence. It is optimizing for **plausibility**, not **truth**.

In [2]:
class MarkovModel:
    def __init__(self, n_gram=1):
        self.n_gram = n_gram
        self.model = {}
        self.words = []

    def fit(self, text_data):
        self.words = text_data.replace('\n', ' ').split()
        self.model = {}
        print(f"Fitting model with N-gram size: {self.n_gram}...")
        for i in tqdm(range(len(self.words) - self.n_gram), desc="Building Chains"):
            # Create the state (the "history" of N words)
            state = tuple(self.words[i : i + self.n_gram])
            # The target (the next word)
            next_word = self.words[i + self.n_gram]

            if state not in self.model:
                self.model[state] = []
            self.model[state].append(next_word)

        print("Training complete.\n")

    def predict(self, length=20, start_words=None):
        if not self.model:
            return "Error: Model not fitted."

        if start_words:
            start_tokens = start_words.split()
            if len(start_tokens) != self.n_gram:
                return f"Error: Start words must contain exactly {self.n_gram} words."
            current_state = tuple(start_tokens)
            if current_state not in self.model:
                print(f"Warning: '{start_words}' not found in training data. Picking random start.")
                current_state = random.choice(list(self.model.keys()))
        else:
            current_state = random.choice(list(self.model.keys()))
        output = list(current_state)
        for _ in range(length):
            if current_state in self.model:
                possible_next_words = self.model[current_state]
                next_word = random.choice(possible_next_words)
                output.append(next_word)
                current_state = tuple(output[-self.n_gram:])
            else:
                break

        return " ".join(output)

    def get_next_word_probabilities(self, input_gram):
        if not self.model:
            return None # Model not fitted
        tokens = input_gram.split()
        if len(tokens) != self.n_gram:
            return None # Wrong N-gram size

        state = tuple(tokens)
        if state not in self.model:
            return [] # State not found (valid n-gram, but no history)

        possible_next_words = self.model[state]
        total_counts = len(possible_next_words)
        counts = Counter(possible_next_words)

        results = []
        for word, count in counts.items():
            prob = (count / total_counts)
            results.append({
                "word": word,
                "probability": prob,
                "count": count,
                "total": total_counts
            })

        return sorted(results, key=lambda x: x['probability'], reverse=True)

In [3]:
# --- Helper to load file ---
def load_text_file(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            text = f.read()
    except FileNotFoundError:
        print(f"File {filename} not found. Using default text.")
        text = """
        Artificial intelligence is the intelligence of machines or software, as opposed to the intelligence of humans or animals.
        It is also the field of study in computer science that develops and studies intelligent machines.
        "AI" may also refer to the machines themselves.
        AI technology is widely used throughout industry, government, and science.
        Some high-profile applications are: advanced web search engines (e.g., Google Search), recommendation systems (used by YouTube, Amazon, and Netflix),
        understanding human speech (such as Siri and Alexa), self-driving cars (e.g., Waymo), generative or creative tools (ChatGPT and AI art),
        and competing at the highest level in strategic games (such as chess and Go).
        """

    # 1. Convert to lower case
    text = text.lower()

    # 2. Remove punctuation
    # Creates a translation table mapping every punctuation char to None
    text = text.translate(str.maketrans('', '', string.punctuation))

    # 3. Normalize whitespace (removes tabs, newlines, and multiple spaces)
    # split() handles \n, \t, and multiple spaces automatically, join() rebuilds the string cleanly
    clean_text = " ".join(text.split())

    return clean_text

def print_probability_table(phrase, probs):
    if probs is None:
        print(f"Error: Could not calculate probabilities for '{phrase}' (Wrong N-gram size or model empty).")
        return

    if len(probs) == 0:
        print(f"State '{phrase}' never appeared in the training text.")
        return

    print(f"--- Probabilities for state: '{phrase}' ---")
    print(f"{'Next Word':<15} | {'Probability':<12} | {'Count':<10}")
    print("-" * 45)

    for item in probs:
        p_percent = item['probability'] * 100
        print(f"{item['word']:<15} | {p_percent:>6.2f}%      | {item['count']}/{item['total']}")
    print("\n")

In [4]:
# --- MAIN EXECUTION ---

# 1. Load Data
# Ensure you have a 'text.txt' file in the same folder, or it will use the default text.
text_content = load_text_file("../../data/ua.txt")

# 2. Define Experiments
n_gram_settings = [1, 2, 4, 8] # N=8 might be too large for small text (overfitting/memorization)
lengths_to_generate = [10, 50]

print(f"--- Loaded Corpus ({len(text_content.split())} words) ---\n")

for n in n_gram_settings:
    # Initialize and Fit
    mk = MarkovModel(n_gram=n)
    mk.fit(text_content)

    print(f"--- Results for N={n} ---")
    for l in lengths_to_generate:
        generated = mk.predict(length=l)
        print(f"[Length {l}]: ... {generated} ...\n")
    print("-" * 60)

--- Loaded Corpus (961 words) ---

Fitting model with N-gram size: 1...


Building Chains: 100%|██████████| 960/960 [00:00<00:00, 1728867.26it/s]


Training complete.

--- Results for N=1 ---
[Length 10]: ... professores cifop atual centro para implantação do quarto reitor eleito joaquim ...

[Length 50]: ... equipa coordenada pelo arquiteto nuno portas alguns dos mais prestigiados arquitetos portugueses de ensino superior de ensino e formação de professores desde então não exploradas pelas instituições de professores cifop atual centro para associação académica e de cursos técnicos superiores profissionais tesp oferecidos os materiais a escola superior de aveiro mais ...

------------------------------------------------------------
Fitting model with N-gram size: 2...


Building Chains: 100%|██████████| 959/959 [00:00<00:00, 1967875.51it/s]


Training complete.

--- Results for N=2 ---
[Length 10]: ... de especialistas nacionais e estrangeiros o horizonte de formação de professores desde ...

[Length 50]: ... criado o curso de estudos do ambiente e também diversos cursos de especialização tecnológica cet agora cursos técnicos superiores profissionais tesp oferecidos os dois mandatos do sétimo reitor da ua manuel antónio assunção compreendidos entre 20102014 e 20142018 são marcados pela aposta num “campus que pensa um campus que sente” e na ...

------------------------------------------------------------
Fitting model with N-gram size: 4...


Building Chains: 100%|██████████| 957/957 [00:00<00:00, 1794344.63it/s]


Training complete.

--- Results for N=4 ---
[Length 10]: ... da educação nacional josé veiga simão o seu discurso de empossamento da primeira comissão ...

[Length 50]: ... projeto cinco anos volvidos sobre a fundação da universidade de aveiro um grupo de estudantes organizouse e a 28 de junho de 1978 criou a associação de estudantes da universidade de aveiro aauav nome que ainda hoje adota a fase de consolidação da universidade decorre na década de 80 em que se definiu o ...

------------------------------------------------------------
Fitting model with N-gram size: 8...


Building Chains: 100%|██████████| 953/953 [00:00<00:00, 1668268.66it/s]

Training complete.

--- Results for N=8 ---
[Length 10]: ... também diversos cursos de formação de professores ciências da natureza matemática inglêsportuguês e francêsportuguês a população estudantil era ...

[Length 50]: ... currículos e metodologias inovadoras desenvolveuse desta forma uma grande área de intervenção da universidade – a educação e formação de professores foi joão evangelista loureiro antigo vicereitor da ua o grande impulsionador deste projeto cinco anos volvidos sobre a fundação da universidade de aveiro um grupo de estudantes organizouse e a 28 de junho de 1978 criou a ...

------------------------------------------------------------


In [5]:
mk = MarkovModel(n_gram=3)
mk.fit(text_content)
mk.predict(length=100, start_words="a população estudantil")

Fitting model with N-gram size: 3...


Building Chains: 100%|██████████| 958/958 [00:00<00:00, 1860251.50it/s]

Training complete.



'a população estudantil era constituída por 338 estudantes e nesse mesmo ano foram construídas as primeiras infraestruturas próprias situadas onde mais tarde seria implantado o campus universitário de santiago viria a ser um contributo de qualidade para o património arquitetónico da cidade hoje podem ser admirados na ua edifícios da autoria de alcino soutinho álvaro siza vieira pedro ramalho luís ramalho josé maria lopo prata eduardo souto moura adalberto dias rebello de andrade jorge kol de carvalho gonçalo byrne e figueiredo dias anualmente visitados por centenas de especialistas nacionais e estrangeiros o horizonte de formação inicial alargouse a áreas inovadoras como o ambiente'

In [6]:
probs_1 = mk.get_next_word_probabilities("a população estudantil")
probs_2 = mk.get_next_word_probabilities("desta forma uma")

# 2. Print Afterwards
print_probability_table("A população estudantil", probs_1)
print_probability_table("desta forma uma", probs_2)

--- Probabilities for state: 'A população estudantil' ---
Next Word       | Probability  | Count     
---------------------------------------------
era             | 100.00%      | 1/1


--- Probabilities for state: 'desta forma uma' ---
Next Word       | Probability  | Count     
---------------------------------------------
grande          | 100.00%      | 1/1




In [7]:
# --- MAIN EXECUTION ---

# 1. Load Data
# Ensure you have a 'text.txt' file in the same folder, or it will use the default text.
text_content = load_text_file("../../data/trump.txt")

# 2. Define Experiments
n_gram_settings = [1, 2, 4, 8] # N=8 might be too large for small text (overfitting/memorization)
lengths_to_generate = [10, 50]

print(f"--- Loaded Corpus ({len(text_content.split())} words) ---\n")

for n in n_gram_settings:
    # Initialize and Fit
    mk = MarkovModel(n_gram=n)
    mk.fit(text_content)

    print(f"--- Results for N={n} ---")
    for l in lengths_to_generate:
        generated = mk.predict(length=l)
        print(f"[Length {l}]: ... {generated} ...\n")
    print("-" * 60)

--- Loaded Corpus (1077 words) ---

Fitting model with N-gram size: 1...


Building Chains: 100%|██████████| 1076/1076 [00:00<00:00, 2409541.43it/s]


Training complete.

--- Results for N=1 ---
[Length 10]: ... our world war two for virtually 100 of contributions from nato ...

[Length 50]: ... americas contribution to the level of that the trump administrations investment in the results of the trump administration to germany the us paying for more than an agreement did the biggest amount on the country according to the military equipment to heavily contested areas alongside british forces in 2024 the government ...

------------------------------------------------------------
Fitting model with N-gram size: 2...


Building Chains: 100%|██████████| 1075/1075 [00:00<00:00, 1592117.51it/s]


Training complete.

--- Results for N=2 ---
[Length 10]: ... per capita casualty rates of the military alliance they didnt pay the ...

[Length 50]: ... as the deployment of us troops however the agreement did not involve a transfer of sovereignty meaning greenland never became us territory is the only member of the total spent by nato countries in 2024 china generated 997 terawatthours from wind that was more than double that of the largest wind farms ...

------------------------------------------------------------
Fitting model with N-gram size: 4...


Building Chains: 100%|██████████| 1073/1073 [00:00<00:00, 1548155.55it/s]


Training complete.

--- Results for N=4 ---
[Length 10]: ... world at gansu which can be seen from space china generates more wind energy ...

[Length 50]: ... for america he said weve secured commitments for a recordbreaking 18 trillion dollars and later on repeated 18 trillion dollars is invested he has made similar claims before in october he said the us had attracted investments worth 17tn £127tn but there is no publicly available evidence to support figures this big a white ...

------------------------------------------------------------
Fitting model with N-gram size: 8...


Building Chains: 100%|██████████| 1069/1069 [00:00<00:00, 1390729.21it/s]

Training complete.

--- Results for N=8 ---
[Length 10]: ... no wind farms trump also criticised wind energy a familiar target which he said was part of a ...

[Length 50]: ... 18 trillion dollars and later on repeated 18 trillion dollars is invested he has made similar claims before in october he said the us had attracted investments worth 17tn £127tn but there is no publicly available evidence to support figures this big a white house website last updated in november aims to track new investment in us manufacturing ...

------------------------------------------------------------


In [10]:
mk = MarkovModel(n_gram=1)
mk.fit(text_content)
mk.predict(length=100, start_words="trump")

Fitting model with N-gram size: 1...


Building Chains: 100%|██████████| 1076/1076 [00:00<00:00, 1523142.46it/s]

Training complete.



'trump total 96tn £71tn the us national security at the website for example the us bases on his administration had attracted investments his administration to be achieved by other large companies in foreign investment in the military equipment to invoke article 5 that contributed troops and infrastructure it states that but the united arab emirates uae the revenues oil and weve secured commitments for weeks trump also singled out the nazis from nato nations contributed troops and infrastructure it was part of the windfall tax on the us allies they the united arab emirates uae the international trade committee said they'

In [11]:
probs_1 = mk.get_next_word_probabilities("trump ")
probs_2 = mk.get_next_word_probabilities("is")

# 2. Print Afterwards
print_probability_table("trump ", probs_1)
print_probability_table("is ", probs_2)

--- Probabilities for state: 'trump ' ---
Next Word       | Probability  | Count     
---------------------------------------------
also            |  23.08%      | 3/13
made            |   7.69%      | 1/13
touched         |   7.69%      | 1/13
has             |   7.69%      | 1/13
is              |   7.69%      | 1/13
claimed         |   7.69%      | 1/13
incorrectly     |   7.69%      | 1/13
secured         |   7.69%      | 1/13
total           |   7.69%      | 1/13
administration  |   7.69%      | 1/13
administrations |   7.69%      | 1/13


--- Probabilities for state: 'is ' ---
Next Word       | Probability  | Count     
---------------------------------------------
the             |  13.33%      | 2/15
estimated       |  13.33%      | 2/15
a               |  13.33%      | 2/15
critical        |   6.67%      | 1/15
talking         |   6.67%      | 1/15
natos           |   6.67%      | 1/15
higher          |   6.67%      | 1/15
paid            |   6.67%      | 1/15
due            